# Unnormalized log transformed: breast, prostate, thyroid

The unnormalized and log transformed breast, prostate and thyroid

In [1]:
from imp import reload
from train_RNA_ResNet import ResNet
from keras import callbacks as cb
from Calibration_Util import FileIO as io
import os
import numpy as np

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/ubuntu/anaconda3/envs/t

In [2]:
rnaNet = ResNet()

# GTEX as source and TCGA as target
source_file = 'unnorm-log-20PC-GTEX-breast-prostate-thyroid.csv'
target_file = 'unnorm-log-20PC-TCGA-breast-prostate-thyroid.csv'
source_path = os.path.join(io.DeepLearningRoot(), 'data/unnorm/breast-prostate-thyroid/' + source_file)
target_path = os.path.join(io.DeepLearningRoot(), 'data/unnorm/breast-prostate-thyroid/' + target_file)

# Make GTEX target and TCGA source
# target_file = 'unnorm-log-20PC-GTEX-breast-prostate-thyroid.csv'
# source_file = 'unnorm-log-20PC-TCGA-breast-prostate-thyroid.csv'
# source_path = os.path.join(io.DeepLearningRoot(), 'data/unnorm/' + source_file)
# target_path = os.path.join(io.DeepLearningRoot(), 'data/unnorm/' + target_file)

rnaNet.load_data(source_path=source_path,
                target_path=target_path)

In [3]:
print("\ngtex = source shape = " + str(rnaNet.source.shape))
print("tcga = target shape = " + str(rnaNet.target.shape))


gtex = source shape = (636, 20)
tcga = target shape = (211, 20)


In [4]:
import CostFunctions as cf
from keras import backend as K
from sklearn.cross_validation import train_test_split

tissue_map = {'breast': 0, 'thyroid':1, 'prostate':2}
tm = lambda t: tissue_map[t]
sample_ratio = 0.75

In [5]:
source_labels = rnaNet.source_df['tissue'].map(tm).values

source = rnaNet.source.astype('float32')
target = rnaNet.target.astype('float32')

target_train_df, target_test_df = train_test_split(rnaNet.target_df, test_size=0.1, random_state=42)

# sort values based on tissue
target_train_df = target_train_df.sort_values(['tissue'])
target_test_df = target_test_df.sort_values(['tissue'])

# extract tissue labels
target_train_labels = target_train_df.loc[:, 'tissue']
target_test_labels = target_test_df.loc[:, 'tissue']

target_train_counts = target_train_df['tissue'].value_counts().sort_index()
target_test_counts = target_test_df['tissue'].value_counts().sort_index()
print("target train counts")
print(target_train_counts)
print("\ntarget validation counts")
print(target_test_counts)

# extract values as numpy array
target_train = target_train_df.loc[:, "PC1":].values
target_test = target_test_df.loc[:, "PC1":].values

print("\ntarget train shape")
print(target_train.shape)
print("\ntarget validate shape")
print(target_test.shape)
print("")

mmd = cf.MMD(source, target)

target train counts
breast      98
prostate    45
thyroid     46
Name: tissue, dtype: int64

target validation counts
breast      12
prostate     3
thyroid      7
Name: tissue, dtype: int64

target train shape
(189, 20)

target validate shape
(22, 20)

setting scales using KNN
[17.76137789145838, 35.522755782916761, 71.045511565833522]
setting all scale weights to 1


In [6]:
ranges = np.zeros((3, 2), dtype='int32')
ranges[0] = [0, target_train_counts[0]]
ranges[1] = [target_train_counts[0], target_train_counts[0] + target_train_counts[1]]
ranges[2] = [ranges[1, 1], target_train_counts[0] + target_train_counts[1] + target_train_counts[2]]
ranges

array([[  0,  98],
       [ 98, 143],
       [143, 189]], dtype=int32)

In [7]:
r = np.zeros((3, 2), dtype='int32')
low = 0
for i in range(target_train_counts.shape[0]):
    high = low+target_train_counts[i]
    print("low = {0}, high = {1}".format(low, high))
    r[i] = [low, high]
    low = high
    
r

low = 0, high = 98
low = 98, high = 143
low = 143, high = 189


array([[  0,  98],
       [ 98, 143],
       [143, 189]], dtype=int32)

In [8]:

K.cast(np.fromiter((x for x in range(r[0, 0],r[0,1])), dtype='int32'), 'int32')

<tf.Tensor 'Cast/x:0' shape=(98,) dtype=int32>

In [9]:
from math import floor
sample_size = floor(0.75*target_train_counts[0])
ix = np.random.randint(low=r[2,0], high=r[2,1], size=sample_size)
target_train[ix, :]

array([[ -1.03357646e+03,   8.11507946e+01,   9.70998504e+00, ...,
          9.06878899e+00,   6.62615902e+00,  -5.37605620e+00],
       [ -1.02171579e+03,   9.86399036e+01,   1.20468447e+01, ...,
         -1.35230529e+01,   2.15144351e-01,   2.90755955e-01],
       [ -1.03502580e+03,   8.99355013e+01,   1.40546385e+01, ...,
         -3.19342676e+00,  -2.72955830e+00,   1.54746845e+00],
       ..., 
       [ -1.04610982e+03,   5.53295151e+01,   7.32740110e+00, ...,
          6.03615839e+00,   3.52418997e+00,  -1.21050271e+01],
       [ -1.03833186e+03,   9.08418915e+01,   1.86168834e+01, ...,
         -1.35970183e+01,   4.98489158e+00,  -4.79683610e+00],
       [ -1.01323577e+03,   9.35982160e+01,   1.26189160e+01, ...,
         -1.34100773e+01,  -7.63272556e+00,   3.23559866e+00]])

In [10]:
import MultiMMD as mMMD

In [12]:
reload(mMMD)
s = K.cast(source, 'float32')
m = mMMD.MultiMMD(s, rnaNet.target_df)


setting scales using KNN
tissue = breast, low = 0, high = 110
tissue = prostate, low = 110, high = 158
tissue = thyroid, low = 158, high = 211

setting scales for tissue 0 breast
[ 21.28275187  42.56550375  85.13100749]

setting scales for tissue 1 prostate
[  28.8851035    57.770207    115.54041399]

setting scales for tissue 2 thyroid
[ 20.23736355  40.47472709  80.94945419]

setting all scale weights to 1

target train counts
breast      98
prostate    45
thyroid     46
Name: tissue, dtype: int64

target validation counts
breast      12
prostate     3
thyroid      7
Name: tissue, dtype: int64

target train shape
(189, 20)

target validate shape
(22, 20)

calculating training ranges
tissue = breast, low = 0, high = 98
tissue = prostate, low = 98, high = 143
tissue = thyroid, low = 143, high = 189

calculating validation ranges
tissue = breast, low = 0, high = 12
tissue = prostate, low = 12, high = 15
tissue = thyroid, low = 15, high = 22


In [13]:
y_true = K.cast(source_labels, 'int32')
y_pred = K.cast(source_labels, 'int32')
K.eval(m.KerasCost(y_true, y_pred))

InvalidArgumentError: You must feed a value for placeholder tensor 'keras_learning_phase' with dtype bool
	 [[Node: keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: add/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_92_add", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'keras_learning_phase', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_n

In [85]:
print(K.eval(K.gather(m.scales, 0)))
print(K.eval(m.scales))

[ 21.40997124  42.81994247  85.63988495]
[[  21.40997124   42.81994247   85.63988495]
 [  30.3833828    60.76676559  121.53353119]
 [  20.22311592   40.44623184   80.89246368]]


In [166]:
t = K.eval(m.scales)
K.cast(t, 'float32')

<tf.Tensor 'Cast_93/x:0' shape=(3, 3) dtype=float32>

In [125]:
for tissue in tissue_map:
    print(tissue_map[tissue])

0
1
2


In [93]:
sample1_low = 0
sample1_high = target_train_counts['breast']-1
sample1_size = int(target_train_counts['breast'] * sample_ratio)
sample1 = K.cast(K.round(K.random_uniform_variable(shape=tuple([sample1_size]), low=sample1_low, 
                                                   high=sample1_high)), 'int32')
sample1_labels = target_train_labels.iloc[K.eval(sample1)]
sample1_target_train = K.gather(target_train, sample1)

[28  7 70 36 79 86 41 82 15 16  9  2 20 61 83  0 26 61 76 16 69 11 23  1 78
 27 63 77 18 65 38  8 84 13 93 18  8 73 95 34  2 91 32 28 43 22  0 75 38 44
 57 15 58 53 88 43  7 88 67 59 55 15 76 12 48 86 13 42 37 25 42  9 60]


In [88]:
sample2_low = target_train_counts['breast']
sample2_high = sample2_low + target_train_counts['prostate']-1
sample2_size = int(target_train_counts['prostate'] * sample_ratio)
sample2 = K.cast(K.round(K.random_uniform_variable(shape=tuple([sample2_size]), low=sample2_low, 
                                                   high=sample2_high)), 'int32')

sample2_labels = target_train_labels.iloc[K.eval(sample2)]
sample2_target_train = K.gather(target_train, sample2)

In [89]:
sample3_low = target_train_counts['breast'] + target_train_counts['prostate']
sample3_high = target_train.shape[0] - 1
sample3_size = int(target_train_counts['thyroid'] * sample_ratio)
sample3 = K.cast(K.round(K.random_uniform_variable(shape=tuple([sample3_size]), low=sample3_low, 
                                                   high=sample3_high)), 'int32')

sample3_labels = target_train_labels.iloc[K.eval(sample3)]
sample3_target_train = K.gather(target_train, sample3)

In [102]:
#calculate the squared distance between x and y
def squaredDistance(X, Y):
    # X is nxd, Y is mxd, returns nxm matrix of all pairwise Euclidean distances
    # broadcasted subtraction, a square, and a sum.
    r = K.expand_dims(X, axis=1)
    return K.sum(K.square(r-Y), axis=-1)

# this will be self.MMDLayer and the labels will be y_true
def kernel(X, Y, weights, scales):
        #expand dist to a 1xnxm tensor where the 1 is broadcastable
        sQdist = K.expand_dims(squaredDistance(X,Y), 0) 
        #expand scales into a px1x1 tensor so we can do an element wise exponential
        scales = K.expand_dims(K.expand_dims(scales, -1), -1)
        #expand scales into a px1x1 tensor so we can do an element wise exponential
        weights = K.expand_dims(K.expand_dims(weights, -1), -1)
        #calculated the kernel for each scale weight on the distance matrix and sum them up
        return K.sum(weights * K.exp(-sQdist / (K.pow(scales, 2))), 0)

def cost(source, target):
        #calculate the 3 MMD terms
        xx = kernel(source, source)
        xy = kernel(source, target)
        yy = kernel(target, target)
        #calculate the bias MMD estimater (cannot be less than 0)
        MMD = K.mean(xx) - 2 * K.mean(xy) + K.mean(yy)
        #return the square root of the MMD because it optimizes better
        return K.sqrt(MMD)

In [160]:
weights = mmd.weights
scales = mmd.scales

source_index1 = np.where(np.isin(source_labels, 0))[0]
source_index2 = np.where(np.isin(source_labels, 1))[0]
source_index3 = np.where(np.isin(source_labels, 2))[0]
source_index1 = []
source1 = source[source_index1]
source2 = source[source_index2]
source3 = source[source_index3]
source2

array([[ -1.04319763e+03,   8.55411682e+01,  -9.16656208e+00, ...,
         -3.96891212e+00,  -5.97061157e+00,  -6.45357990e+00],
       [ -1.05486597e+03,   8.22033157e+01,  -1.86219841e-01, ...,
         -1.55560875e+00,  -4.43829155e+00,  -4.29482794e+00],
       [ -1.05395630e+03,   9.70952759e+01,   8.19949532e+00, ...,
         -7.10550833e+00,   6.40194416e+00,   1.35061234e-01],
       ..., 
       [ -1.04634741e+03,   7.35175476e+01,  -2.88416481e+01, ...,
         -3.55933452e+00,  -4.31093454e+00,   1.45371590e+01],
       [ -1.05143445e+03,   7.97590179e+01,  -1.58335114e+01, ...,
          1.54808321e+01,   5.85429525e+00,   1.13632555e+01],
       [ -1.05030786e+03,   9.67564774e+01,   2.85908461e+00, ...,
         -8.79446220e+00,  -3.47261906e+00,   3.69658440e-01]], dtype=float32)

In [172]:
# calculate MMD for tissue 1
sample1_target = K.cast(sample1_target_train, 'float32')
xx1 = kernel(source1.astype('float32'), source1.astype('float32'), weights, scales)
xy1 = kernel(source1.astype('float32'), sample1_target, weights, scales)
yy1 = kernel(sample1_target, sample1_target, weights, scales)
MMD1 = K.mean(xx1) - 2*K.mean(xy1) + K.mean(yy1)
#MMD1 = K.sqrt(MMD1)

# calculate MMD for tissue 2
sample2_target = K.cast(sample2_target_train, 'float32')
xx2 = kernel(source2.astype('float32'), source2.astype('float32'), weights, scales)
xy2 = kernel(source2.astype('float32'), sample2_target, weights, scales)
yy2 = kernel(sample2_target, sample2_target, weights, scales)
MMD2 = K.mean(xx2) - 2*K.mean(xy2) + K.mean(yy2)
MMD3 = K.sqrt(MMD3)

# calculate MMD for tissue 3
sample3_target = K.cast(sample3_target_train, 'float32')
xx3 = kernel(source3.astype('float32'), source3.astype('float32'), weights, scales)
xy3 = kernel(source3.astype('float32'), sample3_target, weights, scales)
yy3 = kernel(sample3_target, sample3_target, weights, scales)
MMD3 = K.mean(xx3) - 2*K.mean(xy3) + K.mean(yy3)
MMD3 = K.sqrt(MMD3)


x = np.nan_to_num(K.eval(MMD1))
print(x)
x = K.cast(x, 'float32')
K.sqrt(x)
print(K.eval(x))

print(K.eval(MMD1))
print(K.eval(MMD2))
print(K.eval(MMD3))

0.0
0.0
nan
2.17276
1.72843
3.90119


In [ ]:
rnaNet.init_res_net()

In [ ]:
# callbacks=[rnaNet.lrate, cb.EarlyStopping(monitor='val_loss', patience=100, mode='auto')]
# rnaNet.train(epochs=1000, callbacks=callbacks)

In [ ]:
# from plots import scatter_plot, heatmap
# rnaNet.pca()
# %matplotlib inline

In [ ]:
# scatter_plot(rnaNet.source_pca_df, rnaNet.target_pca_df, title="before")
# scatter_plot(rnaNet.calibrated_source_pca_df, rnaNet.target_pca_df, title="after")

In [ ]:
# heatmap(rnaNet.source_df, rnaNet.target_df, title="before")
# heatmap(rnaNet.calibrated_source_df, rnaNet.target_df, title="after")

In [ ]:
# import CostFunctions as cf
# from keras import backend as K

# source = rnaNet.source.astype('float32')
# target = rnaNet.target.astype('float32')
# calibrated_source = rnaNet.calibrated_source.astype('float32')

# mmd = cf.MMD(source, target, MMDTargetSampleSize=target.shape[0], n_neighbors=10)
# mmd_before = K.eval(mmd.cost(source, target))
# mmd_after = K.eval(mmd.cost(calibrated_source, target))

# print("MMD before: %0.10f" % mmd_before)
# print("MMD after: %0.10f" % mmd_after)

In [ ]:
# save_file = 'calibrated-unnorm-log-20PC-GTEX-breast-prostate-thyroid.csv'
# save_path = os.path.join(io.DeepLearningRoot(), 'data/unnorm/breast-prostate-thyroid/' + save_file)

# rnaNet.save_calibrated(path=save_path)